In [49]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import Word2Vec, CoherenceModel
from gensim.models.keyedvectors import KeyedVectors
import mlflow, os, nltk, random, re
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize
from collections import Counter
import pyLDAvis
import pyLDAvis.gensim
import nltk
from nltk import pos_tag
from nltk.corpus import wordnet, stopwords, words
from nltk.stem import WordNetLemmatizer
from pprint import pprint

In [50]:
nltk.download('words')
nltk.download('punkt')
nltk.download('wordnet')

stop_words = stopwords.words('english')
vocabulary = words.words()

[nltk_data] Downloading package words to
[nltk_data]     /Users/thanawatthongpia/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/thanawatthongpia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/thanawatthongpia/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [51]:
# Convert GloVe vectors to word2vec format
# from gensim.scripts.glove2word2vec import glove2word2vec

# glove_file = "src/glove.42B.300d.txt"
# temp_word2vec_file = "src/w2v.42B.300d.txt"
# glove2word2vec(glove_file, temp_word2vec_file)

In [52]:
def preprocess_text(sentences):

    lemmatizer = WordNetLemmatizer()
    lemmatized_sentences = []
    bigram = gensim.models.Phrases(sentences, min_count=5, threshold=100)
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    
    data_words_nostops = [[word for word in doc if word not in stop_words] for doc in sentences]
    # data_words_bigrams = [bigram_mod[doc] for doc in data_words_nostops]

    for sentence in data_words_nostops:
        lemmatized_tokens = [lemmatizer.lemmatize(token, wordnet.VERB) for token in sentence]
        lemmatized_sentences.append(lemmatized_tokens)
    return lemmatized_sentences

In [53]:
def import_text_to_list_of_lists(file_path, delimiter='\t'):
    result = []
    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()  # Remove leading/trailing whitespace and newlines
            sublist = line.split(delimiter)
            result.append(sublist)
    return result

In [54]:
def CreateW2V_Model(w2v_file_name, pre_train_file, vectorsize=300, window=5, min_count=1, workers=4):

    folder_path = 'src'
    pre_train_file_path = os.path.join(folder_path, pre_train_file)
    w2v_file_path = os.path.join(folder_path, w2v_file_name)
    RUN_NAME = "Create Word2Vec Model"

    if mlflow.active_run():
        mlflow.end_run()
    
    with mlflow.start_run(run_name=RUN_NAME):
        # Check exist model
        if os.path.isfile(pre_train_file_path):
            # Load word vectors
            GloVe_vectors = KeyedVectors.load(pre_train_file_path)
        else:
            GloVe_vectors = KeyedVectors.load_word2vec_format(w2v_file_path, binary=False)

        # Initialize a Gensim Word2Vec model
        model = Word2Vec(vector_size=vectorsize, window=window, min_count=min_count, workers=workers)
        # Set the vocabulary and vectors of the model
        model.build_vocab_from_freq(GloVe_vectors.key_to_index)
        model.wv.key_to_index = GloVe_vectors.key_to_index
        model.wv.vectors = GloVe_vectors.vectors
        model.wv.index_to_key = GloVe_vectors.index_to_key
        
        if not os.path.isfile(pre_train_file_path):
            model.wv.save(pre_train_file_path) 

        # Log parameters
        mlflow.log_param("vectorsize", vectorsize)
        mlflow.log_param("window", window)
        mlflow.log_param("min_count", min_count)
        mlflow.log_param("workers", workers)

        # Log model artifact
        mlflow.log_artifact(pre_train_file_path)
        
        return model

In [55]:
def test_Accuracy(model , test_file_path):
    with open(test_file_path, 'r') as file:
        lines = file.readlines()
    total_questions = 0
    correct_predictions = 0
    random_amount = 1000
    random.seed(42)
    for line in random.sample(lines, random_amount):
        if line.startswith(':'):
            continue  # Skip comment lines and category headers
        question, expected_answer = [word.lower() for word in line.split()[:2]]
        choices = [word.lower() for word in line.split()[2:]]
        try:
            predicted_answer = model.wv.most_similar(positive=[question, choices[1]], negative=[expected_answer])[0][0]
            if predicted_answer == choices[0]:
                correct_predictions += 1
        except KeyError:
            continue  # Skip questions with out-of-vocabulary words

        total_questions += 1

    accuracy = correct_predictions / total_questions * 100
    return accuracy

In [56]:
def train_with_corpus(model, RUN_NAME, new_corpus, epochs):
    
    if mlflow.active_run():
        mlflow.end_run()
        
    with mlflow.start_run(run_name=RUN_NAME):
        # Update the vocabulary with new words
        model.build_vocab(new_corpus, update=True)
        model.train(new_corpus, total_examples=len(new_corpus), epochs=epochs)

        # Log parameters
        mlflow.log_param("epochs", epochs)
        # Log model artifact
        mlflow.sklearn.log_model(model, "word2vec_model")
        # accuracy = test_Accuracy(model,'src/questions-words.txt' )
        # mlflow.log_metric("accuracy", accuracy)
        return model

In [57]:
def plot_vec(model):
    
    vocabulary = words.words()
    all_words = []
    vectors = []
    vocabs = list(set(vocabulary) & set(model.wv.key_to_index.keys()))
    random.seed(42)
    random_amount = 100
    for word in random.sample(vocabs, random_amount):
        all_words.append(word)
        vectors.append(model.wv[word])
        
    tsne = TSNE(n_components=2, random_state=42)
    vectors_tsne = tsne.fit_transform(np.array(vectors))

    #########   PLOT CHART    #######
    plt.figure(figsize=(10, 10))
    for i, word in enumerate(all_words):
        x, y = vectors_tsne[i]
        plt.scatter(x, y)
        plt.annotate(word, (x, y), alpha=0.5, fontsize=8)

    plt.title("t-SNE Visualization of Word2Vec Embeddings")
    plt.show()

In [58]:
def corpus_topic(corpus,num_topics=10):

    data_lemmatized = preprocess_text(corpus)
    id2word = corpora.Dictionary(data_lemmatized)  

    # Term Document Frequency 
    LDA_corpus = [id2word.doc2bow(text) for text in data_lemmatized]
    lda_model = gensim.models.ldamodel.LdaModel(corpus=LDA_corpus,
                                           id2word=id2word,
                                           num_topics=num_topics, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
    
    # Compute Perplexity
    print('\nPerplexity: ', lda_model.log_perplexity(LDA_corpus))  
    # a measure of how good the model is. lower the better.

    # Compute Coherence Score
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    print('Coherence Score: ', coherence_lda)
    pyLDAvis.enable_notebook()
    vis = pyLDAvis.gensim.prepare(lda_model, LDA_corpus, id2word)
    return vis

In [59]:
model = CreateW2V_Model("w2v.42B.300d.txt",'GloVe.wordvectors')

In [60]:
model.wv.most_similar("apple", topn=5)

[('iphone', 0.7339416146278381),
 ('ipad', 0.7129027247428894),
 ('apples', 0.6433200836181641),
 ('blackberry', 0.6380701661109924),
 ('ipod', 0.6341225504875183)]

In [61]:
# for time in range(5):
#     model = train_with_corpus(model,new_corpus,1000)
#     i+=1
#     model.wv.most_similar("apple", topn=5)

In [62]:
# plot_vec(model)

In [63]:
# test_Accuracy(model,'src/questions-words.txt' )

In [64]:
# scb_corpus = import_text_to_list_of_lists('src/Corpus_SCB_tip4u.txt', delimiter='\t')
techx_corpus = import_text_to_list_of_lists('src/Corpus_TechX_(norm).txt', delimiter='\t')
pointx_corpus = import_text_to_list_of_lists('src/Corpus_PointX.txt', delimiter='\t')

In [65]:
def word_similar(model,key,n):
    try:
        for k, vec_score in model.wv.most_similar(key,topn=n):
            print(f"{k:{10}} {vec_score}")
    except KeyError:
        print(f"{key} not found.")

In [66]:
key = 'scb'
word_similar(model,key,5)

aic        0.3304489850997925
gsc        0.32839545607566833
m&i        0.3262021243572235
laurentian 0.32590141892433167
iob        0.3192133605480194


In [67]:
# don't know why preprocess techx corpus first then kernal will crashed
corpus = [techx_corpus]
train_datasets = [preprocess_text(dataset) for dataset in corpus]

for i,dataset in enumerate(train_datasets):
    runname = f"train round {i+1}"
    TechXmodel = train_with_corpus(model,runname,dataset,epochs=100)

2023/07/25 16:09:50 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
/Users/thanawatthongpia/anaconda3/lib/python3.10/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/Users/thanawatthongpia/anaconda3/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
/Users/thanawatthongpia/anaconda3/lib/python3.10/importlib/__init__.py:126: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Us

In [68]:
word_similar(model,key,5)

cardx      0.9925256967544556
techxs     0.9922922849655151
scbtechx   0.9922853112220764
scbx       0.992121696472168
thaicharoen 0.9920690655708313


In [69]:
corpus_topic(preprocess_text(techx_corpus))


Perplexity:  -6.551750502379153
Coherence Score:  0.5097149008739016


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.311814 -0.110606       1        1  20.617615
9     -0.044819  0.292882       2        1  20.393748
1     -0.288021  0.066043       3        1  14.553185
5      0.014427 -0.077747       4        1   9.558940
4      0.120783  0.171756       5        1   8.442187
2      0.143694 -0.065869       6        1   7.779775
7      0.132666  0.137600       7        1   7.631406
8      0.104699 -0.131731       8        1   4.355407
6      0.029777 -0.158046       9        1   3.917433
0      0.098607 -0.124282      10        1   2.750305, topic_info=              Term        Freq       Total Category  logprob  loglift
18             scb  585.000000  585.000000  Default  30.0000  30.0000
858         pointx  699.000000  699.000000  Default  29.0000  29.0000
1126         point  575.000000  575.000000  Default  28.0000  28.0000
21             use  382.000000  382.000000  Default  27.0000  27.0000
822          techx  326.000000  326.000000  Default  26.0000  26.0000
...            ...         ...         ...      ...      ...      ...
1677  periodsummer    5.039218    5.924727  Topic10  -5.0506   3.4316
1678         guess    5.038049    5.923657  Topic10  -5.0508   3.4315
1594   opportunity   10.535204   24.256168  Topic10  -4.3131   2.7595
40           apply   15.366446   70.860228  Topic10  -3.9357   2.0649
92          follow    9.938456   54.742120  Topic10  -4.3714   1.8872

[419 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
937       3  0.924722       ability
65        5  0.970989        access
781       4  0.972904        accord
1125      3  0.993951  accumulation
2016      9  0.812057    accuracies
...     ...       ...           ...
608       6  0.274410          work
1129      3  0.996710         world
1246      3  0.262268         worth
1246      8  0.711871         worth
1363      7  0.981458         years

[438 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 10, 2, 6, 5, 3, 8, 9, 7, 1])

In [70]:
corpus_topic(preprocess_text(pointx_corpus))


Perplexity:  -5.002372597595141
Coherence Score:  0.6499014154296792


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.240995  0.122344       1        1  24.804117
2      0.161659  0.111441       2        1  15.054724
1     -0.160022 -0.223341       3        1  11.417325
4     -0.002594  0.128302       4        1  10.357153
9      0.287387  0.002685       5        1   9.180922
8      0.033690 -0.182296       6        1   8.844756
5     -0.131510  0.152089       7        1   8.638766
6     -0.064550 -0.114218       8        1   4.546301
7      0.127443 -0.042826       9        1   4.047838
3     -0.010508  0.045822      10        1   3.108097, topic_info=        Term        Freq       Total Category  logprob  loglift
23   special   82.000000   82.000000  Default  30.0000  30.0000
70     score   68.000000   68.000000  Default  29.0000  29.0000
232     june   55.000000   55.000000  Default  28.0000  28.0000
42     point  196.000000  196.000000  Default  27.0000  27.0000
50       use   56.000000   56.000000  Default  26.0000  26.0000
..       ...         ...         ...      ...      ...      ...
69   receive    2.622566   64.203375  Topic10  -3.4229   0.2733
70     score    1.116896   68.990352  Topic10  -4.2765  -0.6523
42     point    0.932262  196.740416  Topic10  -4.4572  -1.8809
215     line    0.368023    7.393809  Topic10  -5.3867   0.4709
216      man    0.368023    7.073225  Topic10  -5.3867   0.5152

[407 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
227       1  0.630261      accord
116       4  0.937554  accordance
107       1  0.347427     account
107       2  0.138971     account
107      10  0.416913     account
...     ...       ...         ...
272       3  0.782119   wednesday
311       1  0.735850        week
131       4  0.531570      within
52        6  0.826486       world
224       3  0.949549        year

[367 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 2, 5, 10, 9, 6, 7, 8, 4])

In [71]:
TechXmodel.wv.most_similar('techx',topn=10)

[('nigelvaz', 0.9916048049926758),
 ('scbx', 0.9914949536323547),
 ('suwanprateeb', 0.9911962747573853),
 ('anddigital', 0.9911912083625793),
 ('trirat', 0.9911136627197266),
 ('ispaving', 0.9910975694656372),
 ('developand', 0.9910673499107361),
 ('ofhelping', 0.9910535216331482),
 ('vatsun', 0.9909147620201111),
 ('jatupron', 0.9908701181411743)]